In [ ]:
import numpy as np
from scipy.integrate import dblquad,quad,nquad
import math
from functools import lru_cache
from multiprocessing import Pool


me=0.511
e=0.303
Z=57
l=10
# 这里sigma是坐标空间的
sigma_perp=10
sigma_z=5
P_z=5
b_perp=np.array([0,50,0])

# 定义 2x2 单位矩阵和泡利矩阵
I_2 = np.eye(2, dtype=complex)
I_4 = np.eye(4, dtype=complex)
sigma1 = np.array([[0, 1], [1, 0]], dtype=complex)
sigma2 = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma3 = np.array([[1, 0], [0, -1]], dtype=complex)

# 定义 Gamma 矩阵
gamma0 = np.block([[I_2, np.zeros((2, 2))], [np.zeros((2, 2)), -I_2]])
gamma1 = np.block([[np.zeros((2, 2)), sigma1], [-sigma1, np.zeros((2, 2))]])
gamma2 = np.block([[np.zeros((2, 2)), sigma2], [-sigma2, np.zeros((2, 2))]])
gamma3 = np.block([[np.zeros((2, 2)), sigma3], [-sigma3, np.zeros((2, 2))]])

@lru_cache(maxsize=1000)
def w(theta,phi,s):
    return np.array([[np.exp(-1j*phi/2)*np.cos((theta+(1/2-s)*np.pi)/2)], [np.exp(1j*phi/2)*np.sin((theta+(1/2-s)*np.pi)/2)]], dtype=complex)

@lru_cache(maxsize=1000)
def w_dagger(theta,phi,s):
    return np.array([np.exp(1j*phi/2)*np.cos((theta+(1/2-s)*np.pi)/2), np.exp(-1j*phi/2)*np.sin((theta+(1/2-s)*np.pi)/2)], dtype=complex)

@lru_cache(maxsize=1000)
def u(epsilon, theta, phi, s):
    res= np.block([[np.sqrt(epsilon + me)*w(theta,phi,s)],[2*np.sqrt(epsilon - me)*s*w(theta,phi,s)]])
    return res  

@lru_cache(maxsize=1000)
def u_f(epsilon_f,theta_f,phi_f,s_f):
    res= np.block([np.sqrt(epsilon_f + me)*w_dagger(theta_f,phi_f,s_f),-2*np.sqrt(epsilon_f - me)*s_f*w_dagger(theta_f,phi_f,s_f)])
    return res

@lru_cache(maxsize=1000)
def photon_polar(lamb,theta_k,phi_k):
    return np.array([0,
                     1/np.sqrt(2)*(np.cos(theta_k)*np.cos(phi_k)-1j*lamb*np.sin(phi_k)),
                     1/np.sqrt(2)*(np.cos(theta_k)*np.sin(phi_k)-1j*lamb*np.cos(phi_k)),
                     1/np.sqrt(2)*np.sin(theta_k)], dtype=complex)




@lru_cache(maxsize=1000)
def four_vec(energy,m,theta,phi):
    p=np.sqrt(energy**2-m**2)
    return np.array([energy,p*np.sin(theta)*np.cos(phi),p*np.sin(theta)*np.sin(phi),p*np.cos(theta)])

@lru_cache(maxsize=1000)
def three_vec(energy,m,theta,phi):
    p=np.sqrt(energy**2-m**2)
    return np.array([p*np.sin(theta)*np.cos(phi),p*np.sin(theta)*np.sin(phi),p*np.cos(theta)])
@lru_cache(maxsize=1000)
def two_vec_mod(energy,m,theta,phi):
    p=np.sqrt(energy**2-m**2)
    return np.sqrt(np.array([p*np.sin(theta)*np.cos(phi),p*np.sin(theta)*np.sin(phi)])@np.array([p*np.sin(theta)*np.cos(phi),p*np.sin(theta)*np.sin(phi)]))

@lru_cache(maxsize=1000)
def four_vec_slash(energy,m,theta,phi):
    four_vec_=four_vec(energy,m,theta,phi)
    res= four_vec_[0]*gamma0
    res +=-four_vec_[1]*gamma1
    res +=-four_vec_[2]*gamma2
    res +=-four_vec_[3]*gamma3
    return res

@lru_cache(maxsize=1000)
def photon_polar_slash(lamb,theta_k,phi_k):
    photon_polar_=photon_polar(lamb,theta_k,phi_k)
    res= photon_polar_[0]*gamma0
    res +=-photon_polar_[1]*gamma1
    res +=-photon_polar_[2]*gamma2
    res +=-photon_polar_[3]*gamma3
    return res
        

def M(epsilon,theta,phi,s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb):

    photon_polar_slash_=photon_polar_slash(lamb,theta_k,phi_k)
    four_vec_slash_k=four_vec_slash(omega,0,theta_k,phi_k)
    three_vec_i=three_vec(epsilon,me,theta,phi)
    three_vec_f=three_vec(epsilon_f,me,theta_f,phi_f)
    three_vec_k=three_vec(omega,0,theta_k,phi_k)

    res=(4*np.pi)**(3/2)*Z*e**3/((three_vec_f+three_vec_k-three_vec_i)@(three_vec_f+three_vec_k-three_vec_i))*\
                        u_f(epsilon_f,theta_f,phi_f,s_f)@\
                        (1/(2*(omega*epsilon_f-three_vec_k@three_vec_f))*photon_polar_slash_@\
                        (four_vec_slash(epsilon_f,me,theta_f,phi_f)+four_vec_slash_k+me*I_4)@gamma0\
                        +\
                        1/(-2*(omega*epsilon-three_vec_k@three_vec_i))*gamma0@\
                        (four_vec_slash(epsilon,me,theta,phi)-four_vec_slash_k+me*I_4)@photon_polar_slash_)@\
                        u(epsilon,theta,phi,s)
    return res[0]


def Phi(epsilon,theta,phi,l):
    two_vec_mod_=two_vec_mod(epsilon,me,theta,phi)
    return (4*np.pi)**(3/4)*np.sqrt(epsilon)*sigma_perp*np.sqrt(sigma_z)*(sigma_perp*two_vec_mod_)**(abs(l))/np.sqrt(math.factorial(abs(l)))\
       *np.exp(-sigma_perp**2*two_vec_mod_**2/2)*np.exp(-sigma_z**2*(np.sqrt(epsilon**2-me**2)*np.cos(theta)-P_z)**2/2)*np.exp(1j*l*phi)

@lru_cache(maxsize=None)
def var_L_pre(theta,phi,s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb):
    epsilon=epsilon_f+omega
    res=np.pi/(2*np.pi)**(3)*np.sin(theta)*Phi(epsilon,theta,phi,l)*np.exp(-1j*b_perp@three_vec(epsilon,me,theta,phi))\
        *M(epsilon,theta,phi,s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb)
    return res

# # 通过dblquad进行数值积分（但该方法精度低，且有个别奇异值，如var_L(0.5,1.026,np.pi/2,np.pi/3,0.5,4,np.pi/3,np.pi/3,1)，不推荐使用）
# def var_L(s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb):
#     # 定义积分区间
#     a, b = 0, np.pi  # theta 的积分区间
#     c, d = 0, 2*np.pi  # phi 的积分区间

#     # 使用 dblquad 进行数值积分
#     # 注意：dblquad 要求函数签名为 f(y, x)，因此需要调整参数顺序
#     result_real, error_real = dblquad(lambda phi,theta :var_L_pre(theta,phi,s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb).real, a, b, lambda x: c, lambda x: d)
#     result_imag, error_imag = dblquad(lambda phi,theta :var_L_pre(theta,phi,s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb).imag, a, b, lambda x: c, lambda x: d)

#     # 组合实部和虚部
#     result = result_real + 1j * result_imag
#     error = error_real + 1j * error_imag    # 组合实部和虚部
#     return [result,error]

# # 通过nquad进行数值积分（但该方法精度略低，且速度也慢，不过可能更适合并行，有待进一步研究）
# def var_L(s, epsilon_f, theta_f, phi_f, s_f, omega, theta_k, phi_k, lamb):
#     # 定义被积函数
#     def integrand(theta, phi):
#         # 调用 var_L_pre 函数计算被积函数的值
#         value = var_L_pre(theta, phi, s, epsilon_f, theta_f, phi_f, s_f, omega, theta_k, phi_k, lamb)
#         return value.real, value.imag  # 返回实部和虚部
    
#     # 使用 nquad 进行双重积分
#     # 积分范围：theta [0, pi], phi [0, 2*pi]
#     def real_integrand(theta, phi):
#         return integrand(theta, phi)[0]  # 实部
    
#     def imag_integrand(theta, phi):
#         return integrand(theta, phi)[1]  # 虚部
    
#     # 分别对实部和虚部进行积分
#     result_real, error_real = nquad(real_integrand, ranges=[(0, np.pi), (0, 2 * np.pi)])
#     result_imag, error_imag = nquad(imag_integrand, ranges=[(0, np.pi), (0, 2 * np.pi)])
    
#     # 组合实部和虚部
#     result = result_real + 1j * result_imag
#     error = error_real + 1j * error_imag
    
#     # 返回结果和误差
#     return [result, error]

def var_L(s, epsilon_f, theta_f, phi_f, s_f, omega, theta_k, phi_k, lamb):
    # 定义内层积分函数（对 phi 积分）
    def inner_integral(theta):
        def integrand(phi):
            return var_L_pre(theta, phi, s, epsilon_f, theta_f, phi_f, s_f, omega, theta_k, phi_k, lamb)
        
        # 对 phi 进行积分
        result_real,_ = quad(lambda phi: integrand(phi).real, 0, 2 * np.pi)
        result_imag,_ = quad(lambda phi: integrand(phi).imag, 0, 2 * np.pi)
        return result_real + 1j * result_imag

    # 对外层积分（对 theta 积分）
    def outer_integrand(theta):
        res = inner_integral(theta)
        return res

    # 对 theta 进行积分
    result_real, error_real = quad(lambda theta: outer_integrand(theta).real, 0, np.pi)
    result_imag, error_imag = quad(lambda theta: outer_integrand(theta).imag, 0, np.pi)
    
    # 组合实部和虚部
    result = result_real + 1j * result_imag
    error = error_real + 1j * error_imag
    return [result, error]



omega0=2
# 定义积分域
epsilon_f_min, epsilon_f_max =me+0.1 , np.sqrt(P_z**2+me**2)-omega0+2
theta_f_min, theta_f_max = 0, np.pi
phi_f_min, phi_f_max = 0, 2 * np.pi

# 定义被积函数
def integrand(epsilon_f, theta_f, phi_f):
    var_l = var_L(s=1, epsilon_f=epsilon_f, theta_f=theta_f,
                  phi_f=phi_f, s_f=1, omega=omega0, theta_k=np.pi/3, 
                  phi_k=np.pi/4, lamb=1)[0]
    return abs(var_l)**2  # 模的平方

# 蒙特卡洛积分
def monte_carlo_integration(n_samples=1000):
    # 随机采样
    epsilon_f_samples = np.random.uniform(epsilon_f_min, epsilon_f_max, n_samples)
    theta_f_samples = np.random.uniform(theta_f_min, theta_f_max, n_samples)
    phi_f_samples = np.random.uniform(phi_f_min, phi_f_max, n_samples)

    # 计算被积函数值
    values = np.array([integrand(eps, theta, phi)
                       for eps, theta, phi in zip(epsilon_f_samples, theta_f_samples, phi_f_samples)])
    
    # 积分体积
    volume = (epsilon_f_max - epsilon_f_min) * (theta_f_max - theta_f_min) * (phi_f_max - phi_f_min)
    
    # 近似积分值
    integral = volume * np.mean(values)
    error = volume * np.std(values) / np.sqrt(n_samples)
    return integral, error

# 调用蒙特卡洛积分
result, error = monte_carlo_integration(n_samples=100)
print(f"Result: {result}, Error: {error}")



# print(var_L(0.5,0.512,np.pi/2,np.pi/3,0.5,4,np.pi/3,np.pi/3,1))

# when n= 10 Result: 0.012947428508045294, Error: 0.012273183758930217

#when n= 100 Result: 0.14545343255397986, Error: 0.0728693240484606

#when n =1000 Result: 0.14688442767579576, Error: 0.054463460726583014
# when n=1000 Result: 0.24039260361630851, Error: 0.06094633761819262
# when n=1000  Result: 0.2779734102473671, Error: 0.06653461047417301








